In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import tensorflow as tf
import tensorflow_probability as tfp
import h5py
import random
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tfpl = tfp.layers
tf1 = tf.compat.v1
tfd = tfp.distributions

In [3]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
  # Restrict TensorFlow to only allocate 10GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
hf = h5py.File('/mnt/data/HSC/HSC_v6/step2/127x127/five_band_image127x127_with_metadata.hdf5', 'r')
n = 10000
x = []
y = []
r = random.sample(range(len(hf["specz_redshift"])), n)
oid = [hf['object_id'][i] for i in r]
x = [hf['image'][i] for i in r]
y = np.asarray([hf['specz_redshift'][i] for i in r])
x = np.transpose(x,(0,2,3,1))
max_value = np.max(x)
x_new = np.true_divide(x,max_value)
hf.close()

In [5]:
oid_train, oid_test, x_train, x_test, y_train, y_test = train_test_split(oid, x_new, y, test_size = 0.2)

In [8]:
hf_train = h5py.File('/mnt/data/HSC/HSC_v6/step3/127x127/training_8000_linear.hdf5', 'w')

In [9]:
hf_train.create_dataset('image', data = x_train)
hf_train.create_dataset('specz', data = y_train)
hf_train.create_dataset('object_id', data = oid_train)

<HDF5 dataset "object_id": shape (8000,), type "<i8">

In [10]:
hf_train.close()

In [11]:
hf_test = h5py.File('/mnt/data/HSC/HSC_v6/step3/127x127/testing_2000_linear.hdf5', 'w')

In [12]:
hf_test.create_dataset('image', data = x_test)
hf_test.create_dataset('specz', data = y_test)
hf_test.create_dataset('object_id', data = oid_test)

<HDF5 dataset "object_id": shape (2000,), type "<i8">

In [13]:
hf_test.close()

In [ ]:
# END